In [1]:
import tensorflow as tf
import os
import _pickle as cPickle
import numpy as np

CIFAR_DIR = "C:\\Users\\i076453\\Downloads\\tech\\ml\\cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
def load_data(filename):
    """ read data from data file """
    with open(filename, 'rb') as f:
        data = cPickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

# tensorflow.dataset
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            for item, label in zip(data, labels):
                if label in [0, 1]:  # 只取0,1两类
                    all_data.append(item)
                    all_labels.append(label)
        self._data = np.vstack(all_data)  # 纵向合并成
        self._data = self._data / 127.5 - 1 # 进行归一化 
            
        self._labels = np.hstack(all_labels) # 横向合并
        
        print(self._data.shape)
         print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
        
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples) # 取得10000
        print(p)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch"""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)] 
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
# (10000, 3072) data: 5个文件，每个类别5000张图片，只取两类，就是1/5, 也就是10000
# (10000,) labels: 它是一个一维数组
test_data = CifarData(test_filenames, False)
# (2000, 3072) data: 1个文件，取1/5, 也就是2000 
# (2000,) labels: 

# batch_data, batch_labels = train_data.next_batch(10)
# print(batch_data)
# print(batch_labels)

(10000, 3072)
(10000,)
[3328 2024 8185 ... 2046 9549 6600]
(2000, 3072)
(2000,)


In [3]:
# 构建tensorflow计算图
x = tf.placeholder(tf.float32, [None, 3072]) # data 的placeholder, None表示不确定的
# print(x) Tensor("Placeholder:0", shape=(?, 3072), dtype=float32)
# y的shape是[None]
y = tf.placeholder(tf.int64, [None]) # labels的placeholder
# (3072, 1)
# print(x.get_shape()[-1]) --> 3072
w = tf.get_variable('w', [x.get_shape()[-1], 1],
                    initializer=tf.random_normal_initializer(0,1))  # x.get_shape()[-1] 就是3072
# (1, )
b = tf.get_variable('b', [1], 
                   initializer=tf.constant_initializer(0.0))
# [None, 3072] * [3072, 1] = [None, 1]   matmul 就是 matrix matipule 矩阵相乘
y_ = tf.matmul(x, w) + b

# [None, 1] 将y 变成一个概率值
p_y_1 = tf.nn.sigmoid(y_)

# 因为p_y_1于y比较，y的shape跟它的不同， 所以需要做一个reshape
# 调用完后，就变成[None, 1]这样一个shape
y_reshaped = tf.reshape(y, (-1, 1))

# 类型变换,将其与p_y_1类型统一
y_reshaped_float = tf.cast(y_reshaped, tf.float32)

# 计算损失的目标函数
# reduce_mean求平均值， square是求平方
loss = tf.reduce_mean(tf.square(y_reshaped_float - p_y_1))

# bool 
predict = p_y_1 > 0.5
# [1,0,0,1,0,0,1,1...]  比较后，相等是1，不等是0   （可以理解为：predict为预测值，y_reshaped为真实值）
correct_predict = tf.equal(tf.cast(predict, tf.int64), y_reshaped)
print(correct_predict)

# 准确率
accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float64))

# 有了loss目标函数和准确率后，再去做一个
# AdamOptimizer 是一个梯度下降的变种，1e-3就是10的-3次方0.001 
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)


Instructions for updating:
Colocations handled automatically by placer.
Tensor("Equal:0", shape=(?, 1), dtype=bool)


In [4]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps = 100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict = {
                x: batch_data,
                y: batch_labels
            }
        )
        if (i+1) % 500 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' \
                % (i+1, loss_val, acc_val))
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size) 
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels
                    }
                )
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f' % (i+1, test_acc))

[Train] Step: 500, loss: 0.26149, acc: 0.70000
[2010 5360 9020 ...   69 1033 2769]
[Train] Step: 1000, loss: 0.34989, acc: 0.65000
[3239 3081 1811 ... 4501 5396 5401]
[Train] Step: 1500, loss: 0.19071, acc: 0.80000
[ 870 9866  448 ... 6616 8661 3372]
[Train] Step: 2000, loss: 0.34746, acc: 0.65000
[8464 6339 2443 ...  693 4173 8772]
[Train] Step: 2500, loss: 0.19996, acc: 0.80000
[2005 5082 1774 ... 2373 5141 5278]
[Train] Step: 3000, loss: 0.20339, acc: 0.80000
[7116 8859 8490 ... 9707 3842 6311]
[Train] Step: 3500, loss: 0.11651, acc: 0.85000
[5379 7410 6082 ... 6041 7307 5923]
[Train] Step: 4000, loss: 0.09978, acc: 0.90000
[7857 2610 7418 ... 4823 8091 7419]
[Train] Step: 4500, loss: 0.19998, acc: 0.80000
[9923 7061 3298 ... 2478 7722 7368]
[Train] Step: 5000, loss: 0.20210, acc: 0.80000
(2000, 3072)
(2000,)
[Test] Step: 5000, acc: 0.79000
[1853 1577 1245 ... 8653 1806 2406]
[Train] Step: 5500, loss: 0.17492, acc: 0.80000
[7490 9368 8080 ... 5381 7129  551]
[Train] Step: 6000, loss

[Train] Step: 46000, loss: 0.10012, acc: 0.90000
[8464 7717 6783 ...  970 1313 7024]
[Train] Step: 46500, loss: 0.15515, acc: 0.85000
[3689  306 8441 ... 3303 2573 4145]
[Train] Step: 47000, loss: 0.16465, acc: 0.80000
[ 630 4318 6126 ... 5418 4283 5949]
[Train] Step: 47500, loss: 0.10089, acc: 0.90000
[3454 4207 8904 ... 6990 7444  856]
[Train] Step: 48000, loss: 0.11431, acc: 0.85000
[2247 8966 9241 ... 4223 3557 2025]
[Train] Step: 48500, loss: 0.09999, acc: 0.90000
[9455 6818 7018 ... 7472 2601 6570]
[Train] Step: 49000, loss: 0.15034, acc: 0.85000
[7114 6339 3033 ... 7265 4529 8689]
[Train] Step: 49500, loss: 0.10001, acc: 0.90000
[8317 6250 3030 ... 2460 6746 4717]
[Train] Step: 50000, loss: 0.21141, acc: 0.80000
(2000, 3072)
(2000,)
[Test] Step: 50000, acc: 0.81300
[7664 6399 3994 ... 8105 7044 3580]
[Train] Step: 50500, loss: 0.10013, acc: 0.90000
[6047 8992 7167 ... 1597 7097 3998]
[Train] Step: 51000, loss: 0.10179, acc: 0.90000
[1253 9431  859 ... 2533 8035 6055]
[Train] Ste

[Train] Step: 91500, loss: 0.10003, acc: 0.90000
[4758 2819 7476 ... 8985 3188 4581]
[Train] Step: 92000, loss: 0.21662, acc: 0.75000
[2145 5279  191 ... 5105 6602  688]
[Train] Step: 92500, loss: 0.14955, acc: 0.85000
[9481 4664 9468 ... 3061 5429 2049]
[Train] Step: 93000, loss: 0.11313, acc: 0.90000
[1592 9637 5687 ... 1762 2785 9987]
[Train] Step: 93500, loss: 0.05375, acc: 0.95000
[7093 1679 1215 ... 5058 3325 1847]
[Train] Step: 94000, loss: 0.10000, acc: 0.90000
[5844 3381 4772 ... 5916  697 7259]
[Train] Step: 94500, loss: 0.15176, acc: 0.85000
[2053 8785 7896 ... 4728 3972 9140]
[Train] Step: 95000, loss: 0.10018, acc: 0.90000
(2000, 3072)
(2000,)
[Test] Step: 95000, acc: 0.81200
[3824 2828 5226 ...  626 2877 3368]
[Train] Step: 95500, loss: 0.10031, acc: 0.90000
[3040  806 9414 ... 4001 3831 3733]
[Train] Step: 96000, loss: 0.15015, acc: 0.85000
[3928 4247 5595 ... 9924 3217 7278]
[Train] Step: 96500, loss: 0.10967, acc: 0.90000
[1688 4303 5394 ... 6017 9420 5112]
[Train] Ste